https://dados.cvm.gov.br/dados/FI/DOC/CDA/DADOS

https://dados.cvm.gov.br/dados/FI/CAD/DADOS

In [34]:
import pandas as pd
import os
import chardet
import csv
import logging
import glob


In [35]:
PATH_BASE = r"C:\Users\Gusma\OneDrive\Documentos\Estudos\CVM\CDA"

LIST_BASE = glob.glob(os.path.join(PATH_BASE, "*"))

In [ ]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read(10000)
        result = chardet.detect(raw_data)
        return result['encoding']

def detect_separator(file_path, encoding):
    with open(file_path, 'r', encoding=encoding) as file:
        first_line = file.readline()
        separators = [',', ';', '\t', '|']
        separator_counts = {}
        
        for sep in separators:
            separator_counts[sep] = first_line.count(sep)
        
        return max(separator_counts, key=separator_counts.get)

def process_files(file_list):
    dataframes = []
    df_names = []
    
    for file_path in file_list:
        if not os.path.isfile(file_path) or not file_path.lower().endswith('.csv'):
            continue
            
        try:
            encoding = detect_encoding(file_path)
            separator = detect_separator(file_path, encoding)
            
            try:
                df = pd.read_csv(file_path, encoding=encoding, sep=separator, low_memory=False)
            except:
                try:
                    df = pd.read_csv(file_path, encoding=encoding, sep=separator, on_bad_lines='skip', low_memory=False)
                except:
                    df = pd.read_csv(file_path, encoding=encoding, sep=separator, engine='python', on_bad_lines='skip')
            
            # Nome do arquivo sem extensão
            file_name = os.path.splitext(os.path.basename(file_path))[0]
            
            dataframes.append(df)
            df_names.append(file_name)
                
        except:
            pass
    
    return dataframes, df_names

In [37]:
def renomear_colunas_ativo():
    mapeamento = {
        'BLC_1': 'CD_ISIN',
        'BLC_2': 'NM_FUNDO_CLASSE_SUBCLASSE_COTA',
        'BLC_3': 'DS_SWAP',
        'BLC_4': 'CD_ATIVO',
        'BLC_5': 'CD_INDEXADOR_POSFX',
        'BLC_6': 'CD_INDEXADOR_POSFX',
        'BLC_7': 'CD_ATIVO_BV_MERC',
        'BLC_8': 'DS_ATIVO'
    }
    
    colunas_finais = ['DATA', 'CNPJ_FUNDO_CLASSE', 'DENOM_SOCIAL', 'TP_APLIC', 'TP_ATIVO', 'ATIVO', 
                      'QT_VENDA_NEGOC', 'VL_VENDA_NEGOC', 'QT_AQUIS_NEGOC', 'VL_AQUIS_NEGOC', 
                      'QT_POS_FINAL', 'VL_MERC_POS_FINAL', 'VL_CUSTO_POS_FINAL']
    
    for var_name in vars_criadas:
        df = globals()[var_name]
        
        # Renomeia coluna para ATIVO
        for blc, coluna_original in mapeamento.items():
            if blc in var_name and coluna_original in df.columns:
                df.rename(columns={coluna_original: 'ATIVO'}, inplace=True)
                break
        
        # Extrai data do nome do arquivo (formato 202508 -> 01/08/2025)
        ano_mes = var_name[-6:]  # Últimos 6 caracteres
        ano = ano_mes[:4]
        mes = ano_mes[4:6]
        data_formatada = f"01/{mes}/{ano}"
        
        # Adiciona coluna DATA
        df.insert(0, 'DATA', data_formatada)
        
        # Filtra apenas as colunas desejadas
        colunas_existentes = [col for col in colunas_finais if col in df.columns]
        globals()[var_name] = df[colunas_existentes]

renomear_colunas_ativo()

In [38]:
def concatenar_dfs_blc():
    dfs_blc = []
    
    for var_name in vars_criadas:
        if 'BLC' in var_name:
            df = globals()[var_name]
            dfs_blc.append(df)
    
    if dfs_blc:
        df_cda_fi_BLC = pd.concat(dfs_blc, ignore_index=True)
        globals()['df_cda_fi_BLC'] = df_cda_fi_BLC

concatenar_dfs_blc()

In [ ]:
# Processa os arquivos
dfs, nomes_dfs = process_files(LIST_BASE)

# Cria variáveis com nome do arquivo
for i, nome in enumerate(nomes_dfs):
    nome_var = f"df_{nome}".replace('-', '_').replace(' ', '_')
    globals()[nome_var] = dfs[i]

# Lista os nomes das variáveis criadas
vars_criadas = [f"df_{nome}".replace('-', '_').replace(' ', '_') for nome in nomes_dfs]

renomear_colunas_ativo()
concatenar_dfs_blc()